In [1]:
import torch
from torchvision import transforms
from PIL import Image
from enum import Enum
import time
import torchvision.models as models
import datetime
import pytz
import os
import logging

logging.basicConfig(level=logging.DEBUG, filename="cpu_logs_exe_mobilenet_max.txt", filemode="a+",format="")

def read_classes():
    """
    Load the ImageNet class names.
    """
    with open("imagenet-classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    return categories

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
logging.info(f"Device: {device}")

In [2]:
use_cuda = torch.cuda.is_available()
use_cuda

True

In [ ]:
import sys
    
def load_model(model_name):
    if model_name == "resnet50":
        model = models.resnet50(weights= models.ResNet50_Weights.IMAGENET1K_V1)    
    elif model_name == "mobilenetv3":
        model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1)
    elif model_name == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    elif model_name == "resnext50":
        model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
    elif model_name == "squeezenet":
        model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)
    else:
        print(f"Model {model_name} not supported yet.")
        sys.exit(1)
    print("After loading model: ",datetime.datetime.now(pytz.timezone('Europe/London')))
    logging.info(f"After loading model: {datetime.datetime.now(pytz.timezone('Europe/London'))}")
    print(f"Loaded PyTorch {model_name} pretrained on ImageNet")
    logging.info(f"Loaded PyTorch {model_name} pretrained on ImageNet")
    return model
    
print("Before loading model: ",datetime.datetime.now(pytz.timezone('Europe/London')))
logging.info(f"Before loading model: {datetime.datetime.now(pytz.timezone('Europe/London'))}")
# Load the pre-trained Mobilenetv3 model
model = load_model("mobilenetv3")
model = model.to(device)
model.eval()  # Set the model to evaluation mode
categories = read_classes()

Before loading model:  2024-11-01 19:34:13.200190+00:00
After loading model:  2024-11-01 19:34:13.712775+00:00
Loaded PyTorch mobilenetv3 pretrained on ImageNet


In [3]:

def preprocess_image(image_path, target_size):
    print("Loading image: ",datetime.datetime.now(pytz.timezone('Europe/London')))
    logging.info(f"Loading image: {datetime.datetime.now(pytz.timezone('Europe/London'))}")
    input_image = Image.open(image_path).convert('RGB')
    if input_image is None:
        print(f"Failed to load image: {image_path}")
        return None, None
    
    logging.info(f"Image transformation: {datetime.datetime.now(pytz.timezone('Europe/London'))}")
    # Define the image transformation pipeline
    width, height = input_image.size  # Height x Width
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Apply transformations to the input image
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)
    return input_batch, width, height

In [ ]:
# Make the prediction

def classify_images(model, fps):
    target_size = (224, 224)
    # interval = 1 / fps
    images_dir = "./Images"
    loop_run_count = 0
    for loop_run_count in range(1,101):
        print(f"Loop run count: {loop_run_count}")
        logging.info(f"Loop run count: {loop_run_count}")
        for image_file in sorted(os.listdir(images_dir)):
            image_path = os.path.join(images_dir, image_file)
            
            if not image_file.endswith(('.jpg', '.jpeg', '.png')):
                continue
            start_time = time.time()
            img, width, height = preprocess_image(image_path, target_size)
            if img is None:
                continue

            with torch.no_grad():
                img = img.to(device)
                print("During prediction: ", datetime.datetime.now(pytz.timezone('Europe/London')))
                logging.info(f"During prediction: {datetime.datetime.now(pytz.timezone('Europe/London'))}")
                predictions = model(img)
                end_time = time.time()
                predicted_class = predictions.argmax(dim=1).item() 

            # Get the predicted class index
            # get the softmax probabilities
            probabilities = torch.nn.functional.softmax(predictions[0], dim=0)
            # check the top category that are predicted
            top5_prob, top5_catid = torch.topk(probabilities, 1)

            print(f"Image: {image_file}")
            logging.info(f"Image: {image_file}") 
            logging.info(f"Resolution: {height}x{width}")
            # logging.info(f"Set FPS: {fps}")
            # Print the predicted class label and inference time
            inference_time = end_time - start_time
            # latency  is total inference time divided with actual runs without the initial model inference time
            if device == "cuda":
                latency = inference_time / 8 # divided with 8 as initial 2 runs out of 10 has higher inference time for model loading
            else: 
                latency = inference_time / 10
            
            # print(f'Inference time: {inference_time:.4f} seconds')
            # print(f"Accuracy: {top5_prob[0].item()*100:.3f}%")
            # print(f"Argmax Pred class:{categories[predicted_class]}")
            # print(f"TopK Predicted class:{categories[top5_catid[0]]}")
            # print(f'Latency: {latency:.4f} seconds')
            print("")

            logging.info(f'Inference time: {inference_time:.4f} seconds')
            logging.info(f"Accuracy: {top5_prob[0].item()*100:.3f}%")
            logging.info(f"Argmax Pred class:{categories[predicted_class]}")
            logging.info(f"TopK Predicted class:{categories[top5_catid[0]]}")
            logging.info(f'Latency: {latency:.4f} seconds')
            logging.info("")
            # if inference_time < interval:
            #     time.sleep(interval - inference_time)

In [5]:
# for i in range(1, 11):
#     logging.info("")
#     inference_time = classify_images(model, i)

inference_time = classify_images(model, 1)

Loop run count: 1
Loading image:  2024-11-01 19:34:36.027812+00:00
During prediction:  2024-11-01 19:34:36.342373+00:00
Image: 1.jpg

Loading image:  2024-11-01 19:34:39.232733+00:00
During prediction:  2024-11-01 19:34:39.267367+00:00
Image: cat_0.jpg

Loading image:  2024-11-01 19:34:41.988009+00:00
During prediction:  2024-11-01 19:34:42.009388+00:00
Image: cat_1.jpg

Loading image:  2024-11-01 19:34:44.887617+00:00
During prediction:  2024-11-01 19:34:44.924685+00:00
Image: cat_2.jpg

Loading image:  2024-11-01 19:34:47.823971+00:00
During prediction:  2024-11-01 19:34:47.859652+00:00
Image: dog_0.jpg

Loading image:  2024-11-01 19:34:50.538359+00:00
During prediction:  2024-11-01 19:34:50.574891+00:00
Image: dog_1.jpg

Loop run count: 2
Loading image:  2024-11-01 19:34:53.362184+00:00
During prediction:  2024-11-01 19:34:53.395851+00:00
Image: 1.jpg

Loading image:  2024-11-01 19:34:56.101131+00:00
During prediction:  2024-11-01 19:34:56.143404+00:00
Image: cat_0.jpg

Loading imag

KeyboardInterrupt: 

In [ ]:
# (1) FLOPS, the lower the better, 
# (2) number of parameters, the lower the better, 
# (3) fps, the higher the better, 
# (4) latency, the lower the better
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')
logging.info(f'Total trainable parameters: {total_params}')

Total trainable parameters: 11689512


In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained Mobilenetv3 model
model = load_model("resnet18")

# Define input size (224x224x3) for Mobilenetv3
input_size = (224, 224, 3)

# Function to calculate FLOPs for the model
def count_flops(model, input_size):
    # Define a tensor of appropriate size
    input_tensor = torch.randn(1, *input_size)
    # Switch to evaluation mode
    model.eval()
    
    # Move model to appropriate device
    device = "cuda"
    input_tensor = input_tensor.to(device)
    model.to(device)
    
    # Iterate through model's layers
    flops = 0
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            # For convolutional layers
            output_size = (input_size[0] - module.kernel_size[0] + 2 * module.padding[0]) // module.stride[0] + 1
            output_size = (output_size - module.kernel_size[1] + 2 * module.padding[1]) // module.stride[1] + 1
            flops += module.in_channels * module.out_channels * module.kernel_size[0] * module.kernel_size[1] * output_size * output_size
            input_size = (output_size, output_size, module.out_channels)
        elif isinstance(module, torch.nn.MaxPool2d):
            # For max pooling layers
            output_size = (input_size[0] - module.kernel_size) // module.stride + 1
            flops += input_size[2] * output_size * output_size
            input_size = (output_size, output_size, input_size[2])
        elif isinstance(module, torch.nn.Linear):
            # For fully connected layers
            flops += module.in_features * module.out_features
            input_size = (module.out_features,)
    
    return flops

# Calculate FLOPs
total_flops = count_flops(model, input_size)
print("Total FLOPs:", total_flops)
logging.info("Total FLOPs:", total_flops)

Loaded PyTorch resnet18 pretrained on ImageNet
Total FLOPs: 160093760


Total FLOPs: 29234304
Before loading model:  2024-10-18 17:57:17.025792+01:00

0: 1.jpg

Loading image:  2024-10-18 17:57:31.158511+01:00
Image transformation:  2024-10-18 17:57:35.574077+01:00
During prediction:  2024-10-18 17:57:39.639212+01:00
Inference time: 2.9779 seconds
Accuracy: 40.332%
Predicted class:Italian greyhound

datetime.datetime(2024, 10, 18, 17, 57, 17, 948191)
datetime.datetime(2024, 10, 18, 17, 57, 18, 950298)
datetime.datetime(2024, 10, 18, 17, 57, 19, 952551)
datetime.datetime(2024, 10, 18, 17, 57, 20, 942129)
datetime.datetime(2024, 10, 18, 17, 57, 21, 943182)
datetime.datetime(2024, 10, 18, 17, 57, 22, 943864)
datetime.datetime(2024, 10, 18, 17, 57, 23, 957899)
datetime.datetime(2024, 10, 18, 17, 57, 24, 966308)
datetime.datetime(2024, 10, 18, 17, 57, 25, 954725)
datetime.datetime(2024, 10, 18, 17, 57, 26, 951080)
datetime.datetime(2024, 10, 18, 17, 57, 27, 965039)
datetime.datetime(2024, 10, 18, 17, 57, 28, 964528)
datetime.datetime(2024, 10, 18, 17, 57, 29, 970994)
datetime.datetime(2024, 10, 18, 17, 57, 30, 957246)
datetime.datetime(2024, 10, 18, 17, 57, 31, 966759)
datetime.datetime(2024, 10, 18, 17, 57, 32, 969495)
datetime.datetime(2024, 10, 18, 17, 57, 33, 959996)
datetime.datetime(2024, 10, 18, 17, 57, 34, 972118)
datetime.datetime(2024, 10, 18, 17, 57, 35, 974874)
datetime.datetime(2024, 10, 18, 17, 57, 36, 975562)
datetime.datetime(2024, 10, 18, 17, 57, 37, 976466)
datetime.datetime(2024, 10, 18, 17, 57, 38, 966991)
datetime.datetime(2024, 10, 18, 17, 57, 40, 053279)

Average CPU Temperature: 31.13°C
Average GPU Temperature: 32.02°C
Average Power CPU: 868.96mW
Average Power GPU: 868.96mW
Average Power TOT: 2325.17mW
Average RAM: 0.4688

{'Avg_CPU_temp': 31.130434782608695, 'Avg_GPU_temp': 32.02173913043478, 'Avg_Power_CPU': 868.9565217391304, 'Avg_Power_GPU': 868.9565217391304, 'Avg_CPU_RAM': 0.46881048197927816}

1: cat_0

Loading image:  2024-10-18 17:59:21.789642+01:00
Image transformation:  2024-10-18 17:59:24.607439+01:00
During prediction:  2024-10-18 17:59:27.012594+01:00
Inference time: 2.8112 seconds
Accuracy: 14.485%
Predicted class:water jug

datetime.datetime(2024, 10, 18, 17, 59, 21, 120204)
datetime.datetime(2024, 10, 18, 17, 59, 22, 126424)
datetime.datetime(2024, 10, 18, 17, 59, 23, 124276)
datetime.datetime(2024, 10, 18, 17, 59, 24, 117592)
datetime.datetime(2024, 10, 18, 17, 59, 25, 121873)
datetime.datetime(2024, 10, 18, 17, 59, 26, 135385)
datetime.datetime(2024, 10, 18, 17, 59, 27, 196186)
datetime.datetime(2024, 10, 18, 17, 59, 28, 133128)
datetime.datetime(2024, 10, 18, 17, 59, 29, 181014)

Average CPU Temperature: 31.61°C
Average GPU Temperature: 32.50°C
Average Power CPU: 1231.00mW
Average Power GPU: 1231.00mW
Average Power TOT: 2643.22mW
Average RAM: 0.4709
{'Avg_CPU_temp': 31.61111111111111, 'Avg_GPU_temp': 32.5, 'Avg_Power_CPU': 1231.0, 'Avg_Power_GPU': 1231.0, 'Avg_CPU_RAM': 0.4708981101005223}

2: cat_1
Loading image:  2024-10-18 18:01:04.291168+01:00
Image transformation:  2024-10-18 18:01:06.273660+01:00
During prediction:  2024-10-18 18:01:09.836196+01:00
Inference time: 2.8233 seconds
Accuracy: 53.589%
Predicted class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 18, 01, 04, 259825)
datetime.datetime(2024, 10, 18, 18, 01, 05, 279781)
datetime.datetime(2024, 10, 18, 18, 01, 06, 262401)
datetime.datetime(2024, 10, 18, 18, 01, 07, 274778)
datetime.datetime(2024, 10, 18, 18, 01, 08, 276707)
datetime.datetime(2024, 10, 18, 18, 01, 09, 277619)
datetime.datetime(2024, 10, 18, 18, 01, 10, 316841)
datetime.datetime(2024, 10, 18, 18, 01, 11, 342538)

Average CPU Temperature: 31.69°C
Average GPU Temperature: 32.69°C
Average Power CPU: 1076.88mW
Average Power GPU: 1076.88mW
Average Power TOT: 2453.38mW
Average RAM: 0.4712
{'Avg_CPU_temp': 31.6875, 'Avg_GPU_temp': 32.6875, 'Avg_Power_CPU': 1076.875, 'Avg_Power_GPU': 1076.875, 'Avg_CPU_RAM': 0.47117206669228723}

3: dog_0

Loading image:  2024-10-18 18:03:41.256764+01:00
Image transformation:  2024-10-18 18:03:43.954394+01:00
During prediction:  2024-10-18 18:03:50.639748+01:00
Inference time: 2.9095 seconds
Accuracy: 69.595%
Predicted class:malamute, malemute, Alaskan malamute

datetime.datetime(2024, 10, 18, 18, 03, 41, 482930)
datetime.datetime(2024, 10, 18, 18, 03, 42, 480816)
datetime.datetime(2024, 10, 18, 18, 03, 43, 488559)
datetime.datetime(2024, 10, 18, 18, 03, 44, 482902)
datetime.datetime(2024, 10, 18, 18, 03, 45, 488933)
datetime.datetime(2024, 10, 18, 18, 03, 46, 492824)
datetime.datetime(2024, 10, 18, 18, 03, 47, 505770)
datetime.datetime(2024, 10, 18, 18, 03, 48, 501570)
datetime.datetime(2024, 10, 18, 18, 03, 49, 491754)
datetime.datetime(2024, 10, 18, 18, 03, 50, 501876)
datetime.datetime(2024, 10, 18, 18, 03, 51, 510798)
datetime.datetime(2024, 10, 18, 18, 03, 52, 532689)
datetime.datetime(2024, 10, 18, 18, 03, 53, 498738)

Average CPU Temperature: 31.58°C
Average GPU Temperature: 32.50°C
Average Power CPU: 1027.62mW
Average Power GPU: 1027.62mW
Average Power TOT: 2414.23mW
Average RAM: 0.4707
{'Avg_CPU_temp': 31.576923076923077, 'Avg_GPU_temp': 32.5, 'Avg_Power_CPU': 1027.6153846153845, 'Avg_Power_GPU': 1027.6153846153845, 'Avg_CPU_RAM': 0.4706899283154936}

4: cat_2

Loading image:  2024-10-18 18:05:06.948375+01:00
Image transformation:  2024-10-18 18:05:09.383007+01:00
During prediction:  2024-10-18 18:05:12.125914+01:00
Inference time: 2.8526 seconds
Accuracy: 99.132%
Predicted class:Persian cat

datetime.datetime(2024, 10, 18, 18, 05, 06, 602950)
datetime.datetime(2024, 10, 18, 18, 05, 07, 604978)
datetime.datetime(2024, 10, 18, 18, 05, 08, 603793)
datetime.datetime(2024, 10, 18, 18, 05, 09, 602987)
datetime.datetime(2024, 10, 18, 18, 05, 10, 610692)
datetime.datetime(2024, 10, 18, 18, 05, 11, 607423)
datetime.datetime(2024, 10, 18, 18, 05, 12, 657260)
datetime.datetime(2024, 10, 18, 18, 05, 13, 695027)
datetime.datetime(2024, 10, 18, 18, 05, 14, 633435)

Average CPU Temperature: 32.06°C
Average GPU Temperature: 32.72°C
Average Power CPU: 1279.44mW
Average Power GPU: 1279.44mW
Average Power TOT: 2683.11mW
Average RAM: 0.4706
{'Avg_CPU_temp': 32.05555555555556, 'Avg_GPU_temp': 32.72222222222222, 'Avg_Power_CPU': 1279.4444444444443, 'Avg_Power_GPU': 1279.4444444444443, 'Avg_CPU_RAM': 0.4706216489348307}

5: dog_1

Loading image:  2024-10-18 18:08:15.296827+01:00
Image transformation:  2024-10-18 18:08:18.284938+01:00
During prediction:  2024-10-18 18:08:22.015930+01:00
Inference time: 2.6493 seconds
Accuracy: 81.686%
Predicted class:Newfoundland, Newfoundland dog

datetime.datetime(2024, 10, 18, 18, 08, 15, 866883)
datetime.datetime(2024, 10, 18, 18, 08, 16, 864828)
datetime.datetime(2024, 10, 18, 18, 08, 17, 858168)
datetime.datetime(2024, 10, 18, 18, 08, 18, 875126)
datetime.datetime(2024, 10, 18, 18, 08, 19, 869052)
datetime.datetime(2024, 10, 18, 18, 08, 20, 874626)
datetime.datetime(2024, 10, 18, 18, 08, 21, 874836)
datetime.datetime(2024, 10, 18, 18, 08, 22, 882504)
datetime.datetime(2024, 10, 18, 18, 08, 23, 974059)
datetime.datetime(2024, 10, 18, 18, 08, 24, 877386)

Average CPU Temperature: 31.80°C
Average GPU Temperature: 32.60°C
Average Power CPU: 1156.60mW
Average Power GPU: 1156.60mW
Average Power TOT: 2546.70mW
Average RAM: 0.4706

{'Avg_CPU_temp': 31.8, 'Avg_GPU_temp': 32.6, 'Avg_Power_CPU': 1156.6, 'Avg_Power_GPU': 1156.6, 'Avg_CPU_RAM': 0.4705887801657453}

In [9]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  